In [1]:
import pandas as pd
from getModel_Year_MakeID import getModel_Year_MakeID
from fuzzywuzzy import process
import numpy as np


In [2]:
ME_df = pd.read_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\20191218\\NEW eligible vehicles to add 12.18.2019 per John\'s list.xlsx', dtype = str)
ME_df = ME_df.dropna()
ME_df.Make.unique()

array(['Acura', 'Alfa Romeo', 'Audi', 'Bentley', 'BMW', 'Buick',
       'Cadillac', 'Chevrolet', 'Chrysler', 'Dodge', 'Ferrari', 'FIAT',
       'Ford', 'Genesis', 'Geo', 'GMC', 'Honda', 'Hummer', 'Hyundai',
       'Infiniti', 'Isuzu', 'ISUZU', 'Kia', 'Land Rover', 'Lexus',
       'Lincoln', 'Mazda', 'Mercedes-Benz', 'Mercury', 'MINI',
       'Mitsubishi', 'Nissan', 'Plymouth', 'Pontiac', 'Ram', 'Saab',
       'Saturn', 'Scion', 'Smart', 'Subaru', 'Suzuki', 'Toyota',
       'Volkswagen', 'Volvo'], dtype=object)

In [3]:
ME_df.Make.unique()
import json

import httplib2
import pandas as pd



url = "https://vpic.nhtsa.dot.gov/api/vehicles/GetAllMakes?format=json"
h = httplib2.Http()
_, content = h.request(url, "GET")
result = json.loads(content)
ALL = pd.DataFrame(result["Results"])

Dict = {}
for make in ME_df.Make.unique():
    try:
        a = ALL[ALL["Make_Name"].str.strip().str.lower() == make.lower()]
    except:
        Dict[make.lower()] = 'Empty'
    else:
        Dict[make.lower()] = a.Make_ID.values[0] if len(a.Make_ID)>0 else 'N/A'
Dict
rev_Dict = {}
for key, value in Dict.items():
    key, value = value, key
    rev_Dict[key] = value
rev_Dict

    

{475: 'acura',
 493: 'alfa romeo',
 582: 'audi',
 583: 'bentley',
 452: 'bmw',
 468: 'buick',
 469: 'cadillac',
 467: 'chevrolet',
 477: 'chrysler',
 476: 'dodge',
 603: 'ferrari',
 492: 'fiat',
 460: 'ford',
 5083: 'genesis',
 1146: 'geo',
 472: 'gmc',
 474: 'honda',
 951: 'hummer',
 498: 'hyundai',
 480: 'infiniti',
 542: 'isuzu',
 499: 'kia',
 444: 'land rover',
 515: 'lexus',
 464: 'lincoln',
 473: 'mazda',
 449: 'mercedes-benz',
 465: 'mercury',
 456: 'mini',
 481: 'mitsubishi',
 478: 'nissan',
 2409: 'plymouth',
 536: 'pontiac',
 496: 'ram',
 572: 'saab',
 1056: 'saturn',
 'N/A': 'scion',
 504: 'smart',
 523: 'subaru',
 509: 'suzuki',
 448: 'toyota',
 482: 'volkswagen',
 485: 'volvo'}

In [45]:
def getMatch(Make_id, Year, Dict):
    Rlist = []

    try:
        Option_list = getModel_Year_MakeID(Make_id, Year)
        if len(getModel_Year_MakeID(Make_id, Year)) == 0 :
            raise Exception

    except Exception:
        QC_Make_year = ME_df[
            (ME_df.Make.str.lower() == Dict[Make_id]) & (ME_df.Year == Year)
        ].sort_values(by=["Make", "Year", "Model"])

        Rlist = ["No Data in the Database"] * len(QC_Make_year)
    else:
        QC_Make_year = ME_df[
            (ME_df.Make.str.lower() == Dict[Make_id]) & (ME_df.Year == Year)
        ].sort_values(by=["Make", "Year", "Model"])

        for i in range(len(QC_Make_year.Model)):
            str2Match = str(QC_Make_year.Model.values[i])
            strOptions = Option_list
            Ratios = process.extract(str2Match, strOptions)
            Rlist.append([a + ": " + str(dict(Ratios)[a]) for a in dict(Ratios)])

    return Rlist

def getQCtable_allyear(Dict):
    
    row_number = 1

    QC_Make = pd.DataFrame()
    
    for Make_id in Dict:   
        
        Years = list(ME_df[ME_df.Make.str.lower() == Dict[Make_id]].Year.unique())  
        
        for Year in Years:

            Match_list = getMatch(Make_id, Year,Dict)

            TopMatch = [rec[0].split(":")[0] if len(rec) >0 and type(rec) == list else 'N/A' for rec in Match_list]

            QC_Make_Year = ME_df[
                (ME_df.Make.str.lower() == Dict[Make_id]) & (ME_df.Year == Year)
            ].sort_values(by=["Make", "Year", "Model"])

            QC_Make_Year["TopMatch"] = TopMatch 

            QC_Make_Year["MatchOptions"] = Match_list

            QC_Make_Year["Match"] = np.where(
                QC_Make_Year.Model.str.lower() == QC_Make_Year.TopMatch.str.lower(),
                "Yes",
                "No",
            )

            QC_Make_Year["HighestScore"] = [
                int(rec[0].split(": ")[1]) if type(rec) != str else "N/A"
                for rec in QC_Make_Year["MatchOptions"]
            ]

#             QC_Make_Year["TopMaTchExsting"] = [
#                 np.where(
#                     QC_Make_Year["TopMatch"].values[i].lower()
#                     in QC_Make_Year.Model.str.lower().unique(),
#                     "Yes",
#                     "No",
#                 )
#                 if QC_Make_Year["HighestScore"].values[i] >= 80
#                 else "No"
#                 for i in range(len(QC_Make_Year["TopMatch"]))
#             ]

            QC_Make = pd.concat([QC_Make, QC_Make_Year])
            print(row_number,Make_id,rev_Dict[Make_id],Year)
            row_number += 1

    return QC_Make



In [46]:
Output_df = getQCtable_allyear(rev_Dict)

1 475 acura 2013
2 475 acura 2014
3 475 acura 2015
4 475 acura 2016
5 475 acura 2017
6 475 acura 2012
7 475 acura 2006
8 475 acura 2007
9 475 acura 1996
10 475 acura 1997
11 475 acura 1998
12 475 acura 1999
13 475 acura 2002
14 475 acura 2003
15 475 acura 2009
16 475 acura 2010
17 475 acura 2011
18 475 acura 2000
19 475 acura 2001
20 475 acura 2004
21 475 acura 2005
22 475 acura 2008
23 493 alfa romeo 2001
24 493 alfa romeo 2002
25 493 alfa romeo 2003
26 493 alfa romeo 2016
27 582 audi 2001
28 582 audi 2002
29 583 bentley 2003
30 583 bentley 2004
31 583 bentley 2005
32 583 bentley 2006
33 583 bentley 2007
34 583 bentley 2008
35 583 bentley 2009
36 583 bentley 2010
37 583 bentley 2011
38 583 bentley 2012
39 583 bentley 2013
40 583 bentley 2014
41 583 bentley 2015
42 583 bentley 2016
43 583 bentley 2017
44 452 bmw 2007
45 452 bmw 2008
46 452 bmw 2009
47 452 bmw 2010
48 452 bmw 2011
49 452 bmw 2018
50 452 bmw 1997
51 452 bmw 1998
52 468 buick 2005
53 468 buick 1997
54 468 buick 1998
55 46

413 1056 saturn 2005
414 N/A scion 2013
415 N/A scion 2014
416 N/A scion 2015
417 N/A scion 2016
418 N/A scion 2000
419 N/A scion 2001
420 N/A scion 2002
421 504 smart 2011
422 504 smart 2012
423 504 smart 2013
424 504 smart 2014
425 523 subaru 2019
426 523 subaru 2013
427 523 subaru 1997
428 509 suzuki 2013
429 509 suzuki 2014
430 509 suzuki 2015
431 509 suzuki 2005
432 509 suzuki 2006
433 509 suzuki 2007
434 509 suzuki 2017
435 509 suzuki 2000
436 509 suzuki 2001
437 509 suzuki 2002
438 509 suzuki 2003
439 509 suzuki 2004
440 509 suzuki 2008
441 509 suzuki 2009
442 509 suzuki 2010
443 509 suzuki 2011
444 509 suzuki 2012
445 509 suzuki 2016
446 509 suzuki 2018
447 448 toyota 1997
448 448 toyota 1999
449 448 toyota 2017
450 448 toyota 2018
451 448 toyota 2002
452 448 toyota 2003
453 448 toyota 2004
454 448 toyota 2005
455 448 toyota 2010
456 448 toyota 2000
457 448 toyota 1998
458 448 toyota 2011
459 448 toyota 2001
460 482 volkswagen 2017
461 482 volkswagen 2018
462 482 volkswagen 200

In [51]:
Output_df_sorted = Output_df.sort_values(by=['Make','Model','Year'])
Output_df_sorted.to_excel('C:\\Users\\FFR0103\\Desktop\\My files\\WORK\\Auto part\\QC process\\20191218\\QC_New_Added.xlsx',index = False)

## Test case

In [26]:
getMatch(603, '2000', rev_Dict)

out = dict(list(rev_Dict.items())[13:14]) 
test = {'N/A':'scion'}

getQCtable_allyear(test)

getModel_Year_MakeID(475, '2000')


getModel_Year_MakeID(475,'2000')

# Valid_df = getModel_Year_MakeID(Make_id, Year)
for Make_id in rev_Dict:
    Years = list(ME_df[ME_df.Make.str.lower() == rev_Dict[Make_id]].Year.unique())
    for Year in Years:
        print(Make_id,rev_Dict[Make_id], Year,len(getModel_Year_MakeID(Make_id, Year)))
        

['No Data in the Database']